# 第5回講義 演習

## 課題 1. VAE 言語モデル

#### 言語モデル
言語モデル (Language Model) とは, ある文章 $x$ が生成される過程を$p(x)$として確率的にモデル化したものです. この言語モデルを用いることにより, 文章の尤もらしさを図ったり, またサンプリング$x \sim p(x)$によって尤もらしい文章を生成したりすることが可能になります.

#### Variational Autoencoder (VAE)
VAEは, データ$x$の潜在的な意味を表す潜在変数$z$からの生成過程を確率モデル化したものです.
潜在変数とは, 例えばMNISTの手書き文字であれば文字であることや筆跡などに当たります.

#### VAEを使った言語モデル
Bowman et al. (2016) では, このVAEを用いて言語モデルを表現しています.
これにより, 文章の背後にある特徴 (文章の意図, スタイル, 作者など) を潜在変数$z$としてモデル化することを試みています.

モデルの全体図は次のようになります.

<img src="./figure/bowman_conll2016.png">

出典: S. R. Bowman et al. "Generating Sentences from a Continuous Space". CoNLL. 2016

また, 潜在空間に何かしらの意味をもたせることができるようになるため, $z$を潜在空間内で連続的に遷移させたときに一貫性のある補完ができるようになります.
参考までに, 下の左図は従来の言語モデルからサンプリングしたもの, 右図は今回のVAEを使った言語モデルからサンプリングしたものになります.

<img src="./figure/lm.png" width="700mm">

今回はこのVAEを使った言語モデルを2つのRNN (LSTM) を用いて実装していきます.

In [ ]:
import os
import re
import math
import time

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.utils import shuffle

from torch.nn.utils.rnn import pack_padded_sequence

try:
    from utils import Vocab
except ModuleNotFoundError: # iLect環境
    os.chdir('/root/userspace/chap5')
    from utils import Vocab

np.random.seed(34)
torch.manual_seed(34)

In [ ]:
num_epochs = 5
batch_size = 32

embedding_size = 353 # 単語の埋め込み次元数
hidden_size = 191 # LSTMの隠れ層の次元数
latent_size = 13  # 潜在変数の次元数

min_count = 1

PAD = 0
BOS = 1
EOS = 2
UNK = 3
PAD_TOKEN = '<PAD>'
UNK_TOKEN = '<UNK>'
BOS_TOKEN = '<S>'
EOS_TOKEN = '</S>'

TRAIN_X_PATH = 'data/small_tanaka/train_x.txt'
VALID_X_PATH = 'data/small_tanaka/valid_x.txt'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 1. データの読み込み

今回は日本語のデータセット (https://github.com/odashi/small_parallel_enja) を使用します.

データセットの中身は次のようになっています.

In [ ]:
! head -5 data/small_tanaka/train_x.txt

In [ ]:
def load_data(path, n_data=10e+10):
    data = []
    for i, line in enumerate(open(path, encoding='utf-8')):
        words = line.strip().split()
        data.append(words)
        if i + 1 >= n_data:
            break
    return data

In [ ]:
class DataLoader:
    def __init__(self, data_x, batch_size, shuffle=True):
        """
        :param data_x: list, 文章 (単語IDのリスト) のリスト
        :param batch_size: int, バッチサイズ
        :param shuffle: bool, サンプルの順番をシャッフルするか否か
        """
        self.data_x = data_x
        
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.start_index = 0
        
        self.reset()
    
    def reset(self):
        if self.shuffle:
            self.data_x = shuffle(self.data_x)
        self.start_index = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        # ポインタが最後まで到達したら初期化する
        if self.start_index >= len(self.data_x):
            self.reset()
            raise StopIteration()
        
        # バッチを取得
        batch_x = self.data_x[self.start_index:self.start_index+self.batch_size]
        
        # 系列長で降順にソート
        batch_x = sorted(batch_x, key=lambda x: len(x), reverse=True)
        
        # 系列長を取得
        batch_x = [[BOS] + x + [EOS] for x in batch_x]
        batch_x_lens = [len(x) for x in batch_x]
        
        # <S>, </S>を付与 + 短い系列にパディング
        max_length = max(batch_x_lens)
        batch_x = [x + [PAD] * (max_length - len(x)) for x in batch_x]

        # tensorに変換
        batch_x = torch.tensor(batch_x, dtype=torch.long, device=device)
        batch_x_lens = torch.tensor(batch_x_lens, dtype=torch.long, device=device)
        
        # ポインタを更新する
        self.start_index += self.batch_size
        
        return batch_x, batch_x_lens

In [ ]:
vocab = Vocab({
    PAD_TOKEN: PAD,
    BOS_TOKEN: BOS,
    EOS_TOKEN: EOS,
    UNK_TOKEN: UNK,
}, UNK_TOKEN)

sens_train_X = load_data(TRAIN_X_PATH)
sens_valid_X = load_data(VALID_X_PATH)

vocab.build_vocab(sens_train_X, min_count)

train_X = [vocab.sentence_to_ids(sen) for sen in sens_train_X]
valid_X = [vocab.sentence_to_ids(sen) for sen in sens_valid_X]

vocab_size = len(vocab.word2id)
print('語彙数:', vocab_size)
print('学習用データ数:', len(train_X))
print('検証用データ数:', len(valid_X))

In [ ]:
dataloader_train = DataLoader(train_X, batch_size, shuffle=True)
dataloader_valid = DataLoader(valid_X, batch_size, shuffle=False)

### 2. モデルの定義

推論モデル(Encoder)をLSTM + MLP, 生成モデル(Decoder)をLSTMで実装します.

EncoderはLSTMによって入力文をエンコードした後2つのMLPによってそれぞれガウス分布$\mu$の平均と分散$\sigma^2$を計算し, ガウス分布$\mathcal{N}(\mu, \sigma^2)$からサンプリングされた$z$を出力します.

Decoderは初期状態として潜在変数$z$を受け取り, LSTMによって文を生成(デコード)していきます.

<img src="figure/bowman_conll2016.png">

出典: S. R. Bowman et al. "Generating Sentences from a Continuous Space". CoNLL. 2016

#### 2.1. 損失関数

損失関数は次の式のようになります. 第一項は再構成誤差とよばれ負の対数尤度に当たります. 第二項は正則化項で事後分布$q_\theta(z|x)$を事前分布$p(z)$に近づける役割を果たします.

$$
    \mathcal{L}(\theta;x) = - \mathbb{E}_{q_{\theta}(z|x)} \left[\log{p_{\theta}(x|z)}\right] + D_{\mathrm{KL}}\left(q_{\theta}(z|x)||p(z)\right)
$$

第二項のKLダイバージェンスの式は次のようになります.

$$
    D_{\mathrm{KL}} \left(q_{\phi} (z|x) || p(z) \right) = -\frac{1}{2} \sum^M_{m=1} \left( 1 + \log \sigma^2_m - \mu^2_m - \sigma^2_m \right)
$$

#### 2.2. 学習トリック

##### 2.2.1. KL項のアニーリング

損失関数においてKL項の学習は対数尤度のそれよりも簡単です. そのため上記の式のままだと早い段階でKL項の損失が0になり, 潜在変数$z$を考慮しない普通の言語モデルと同じになってしまいます (posterior collapse) .

これを防ぐため, KL項の大きさを係数$\lambda$によってコントロールします. 学習の初期段階では小さい値(0)を設定し正則化の力を弱め, 潜在変数$z$にできるだけ多くの情報がencodeされるように仕向けます. そして学習ステップが進むにつれて$\lambda$を1に近づけていき, 本来のガウス分布に埋め込まれるようにしていきます.

この関数を`get_kl_weight`として実装します.
$\lambda$の計算の仕方としては, sigmoidやtanh関数を使って滑らかに推移させていく方法や, 0から1に線形に増加させていく方法などがあります. 初出のBowmanの論文では前者を用いており, 最近の論文では後者のほうが多い印象です.

今回は原論文に従って前者で実装を行います.
アニーリングの関数は次のようになります.

In [ ]:
def get_kl_weight(step):
    """step数でアニーリングしたKL項の重みを取得する (0 -> 1)
    """
    return (math.tanh((step - 3500)/1000) + 1) /2

この係数$\lambda$の推移のイメージは次のグラフのようになります.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

xs = np.arange(10000)
ys = np.array([get_kl_weight(x) for x in xs])

plt.plot(xs, ys)
plt.xlabel(r'学習のステップ数')
plt.ylabel(r'KL項の重み')
plt.xlim(0, 10000)
plt.ylim(0, 1.0)

##### 2.2.2. Word dropout

Decoderは潜在変数$z$だけでなく元の文$x$も入力として受け取ります.
そのためDecoderの関数としての表現力が強いと$z$を無視しても学習が進むようになってしまい, 潜在変数$z$を考慮した文生成ができなくなってしまいます.

これを防ぐため, Decoderの入力単語を一定の確率$p$で`<UNK>`に置き換えます.
これにより, 潜在変数$z$をより頼りにして学習を進めるようになります.

In [ ]:
word_drop_rate = 0.38 # 単語を<UNK>に置き換える確率 (論文では 0.38)

実際のDecoderクラス内の該当箇所の実装は次のようになっています.

```python
class Decoder:
    ...
        # 単語を一定確率pで<UNK>に置き換える
        if self.training:
            prob = torch.full_like(x, self.word_drop_rate, dtype=torch.float32).to(device)
            mask = torch.bernoulli(prob)
            x = torch.where(mask == 1, torch.full_like(x, UNK).to(device), x)
    ...
```

#### 2.3. Encoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, latent_size):
        """
        :param vocab_size: int, 入力言語の語彙数
        :param embedding_size: int, 単語埋め込みの次元数
        :param hidden_size: int, LSTMの隠れ層の次元数
        :param latent_size: int, 洗剤変数zの次元数
        """
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers=1, batch_first=True)
        self.linear_m = nn.Linear(hidden_size, latent_size)
        self.linear_v = nn.Linear(hidden_size, latent_size)
    
    def forward(self, x, x_lens=None):
        """
        :param x: tensor, 入力のバッチ, size=(バッチサイズ, 系列長)
        :param x_lens: tensor, 入力の系列長, size=(バッチサイズ, 系列長)
        :return z: tensor, サンプリングした潜在変数, size=(バッチサイズ, latent_size)
        :return mean: tensor, 潜在変数の平均, size=(バッチサイズ, latent_size)
        :return lvar: tensor, 潜在変数のlog分散, size=(バッチサイズ, latent_size)
        :return loss_kl: tensor, KLダイバージェンス, size=()
        """
        x = self.embedding(x) # 単語の埋め込み
        if x_lens is not None:
            x = pack_padded_sequence(x, lengths=x_lens, batch_first=True)
        
        _, (h_T, _) = self.lstm(x)
        
        h_T.squeeze_()
        
        mean = self.linear_m(h_T) # 平均
        lvar = self.linear_v(h_T) # 分散のlog
        std = torch.exp(0.5 * lvar) # 標準偏差
        
        eps = torch.randn(mean.size()).to(device) # 標準正規分布からサンプリング
        z = mean + std * eps # 潜在変数
        
        loss_kl = # WRITE ME # KL divergenceの計算
        
        return z, mean, lvar, loss_kl

#### 2.4. Decoder

Decoderクラスでは次の2種類の関数を実装します.

##### forward関数
潜在変数$z$を元に, RNN (LSTM) により元の文$x$を復元できるよう一単語ずつ出力していきます.
また通常の言語モデルや翻訳のときと同様に, 正解データ$x$の単語をRNN (LSTM) の各時刻の入力とします (Teacher Forcing).

##### sample関数
潜在変数$z$のみを受け取り, RNN (LSTM) により新しい文$\hat{x}$を生成していきます.
また各時刻で単語を出力する際に, softmaxの確率分布からサンプリングするか, 最大値を取る単語を出力するかを`greedy_decoding`オプションにより切り替えます.

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, latent_size, word_drop_rate):
        """
        :param vocab_size: int, 入力単語の語彙数
        :param embedding_size: int, 単語埋め込みの次元数
        :param hidden_size: int, LSTMの隠れ層の次元数
        :param latent_size: int, 潜在変数zの次元数
        :param word_drop_rate: int, 入力単語をunkに置き換える確率
        """
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size+latent_size, hidden_size, num_layers=1, batch_first=True)
        self.linear_h = nn.Linear(latent_size, hidden_size)
        self.linear_c = nn.Linear(latent_size, hidden_size)
        self.out = nn.Linear(hidden_size, vocab_size)
        
        self.hidden_size = hidden_size
        self.word_drop_rate = word_drop_rate
        
        self.word_drop = nn.Dropout(word_drop_rate)
    
    def forward(self, x, z):
        """
        :param x: tensor, 入力単語のバッチ, size=(バッチサイズ, 系列長)
        :param z: tensor, 潜在変数のバッチ, size=(バッチサイズ, latent_size)
        :return y: tensor, Decoderの出力, size=(バッチサイズ, 系列長, 語彙数)
        """
        # 初期状態
        h_0 = self.linear_h(z).unsqueeze(0) # 隠れ層の初期状態, size=(1, バッチサイズ, 隠れ層の次元数)
        c_0 = self.linear_c(z).unsqueeze(0) # cellの初期状態, size=(1, バッチサイズ, 隠れ層の次元数)
        
        # 単語を一定確率pで<UNK>に置き換える
        if self.training:
            prob = torch.full_like(x, self.word_drop_rate, dtype=torch.float32).to(device)
            mask = torch.bernoulli(prob)
            x = torch.where(mask == 1, torch.full_like(x, UNK).to(device), x)
        
        # デコード
        x = self.embedding(x) # 単語の埋め込み
        x = torch.cat([x, z.unsqueeze(1).repeat(1, x.shape[1], 1)], dim=2) # 潜在変数zを毎時刻単語のembeddingにconcatする

        h, (_, _) = self.lstm(x, (h_0, c_0)) # LSTM
        
        y = self.out(h)
        
        return y # (バッチサイズ, 系列長, 語彙数)
    
    def sample(self, z, max_length, greedy_decoding=True):
        """
        :param z: tensor, 潜在変数のバッチ, size=(バッチサイズ, latent_size)
        :param max_length: int, 生成文のmax長
        :return x_hat: tensor, 生成文, size=(バッチサイズ, 系列長)
        """
        # 初期状態
        h_0 = self.linear_h(z).unsqueeze(0) # 隠れ層の初期状態, size=(1, バッチサイズ, 隠れ層の次元数)
        c_0 = self.linear_c(z).unsqueeze(0) # cellの初期状態, size=(1, バッチサイズ, cellの次元数)
        
        # 最初の単語は<S>
        x_0 = torch.full((z.size(0), 1), BOS, dtype=torch.long).to(device) # (バッチサイズ, 1)
        
        z = z.unsqueeze(1) # size=(バッチサイズ, 1, latent_size)
        
        x_tm1, h_tm1, c_tm1 = x_0, h_0, c_0
        
        x_hat = [] # 生成文を格納するlist
        
        flag = np.zeros(x_0.size(0), dtype=bool) # 出力が終わったかどうかのflag
        
        for _ in range(max_length):
            x_t = self.embedding(x_tm1) # 単語の埋め込み, size=(バッチサイズ, 1, embedding_size)
            x_t = torch.cat([x_t, z], dim=2) # 潜在変数と単語埋め込みをconcat, size=(バッチサイズ, 1, embedding_size+latent_size)
            
            _, (h_t, c_t) = self.lstm(x_t, (h_tm1, c_tm1)) # LSTM
            
            y_t = F.softmax(self.out(h_t[0]), dim=-1) # Softmax, size=(バッチサイズ, 語彙数)
            
            if greedy_decoding:
                # 確率の一番高い単語を取得する
                x_t = y_t.argmax(1).unsqueeze(1)
            else:
                # 確率分布からサンプリングする
                x_t = torch.multinomial(y_t, 1)
            
            x_hat.append(x_t)
            
            # t -> t-1
            x_tm1, h_tm1, c_tm1 = x_t, h_t, c_t
            
            # </S>が出力されたらFlagを更新する
            flag_t = (x_t.squeeze().cpu().numpy() == EOS)
            flag = np.logical_or(flag, flag_t)
            
            # すべての系列で</S>が出力されたら終了
            if np.all(flag):
                break
        
        # listからtensorに変換する
        x_hat = torch.cat(x_hat, dim=1)
        
        return x_hat # size=(バッチサイズ, 系列長)

In [ ]:
E_args = {
    'vocab_size': vocab_size,
    'embedding_size': embedding_size,
    'hidden_size': hidden_size,
    'latent_size': latent_size
}

D_args = {
    'vocab_size': vocab_size,
    'embedding_size': embedding_size,
    'hidden_size': hidden_size,
    'latent_size': latent_size,
    'word_drop_rate': word_drop_rate
}

In [ ]:
# cudnnのエラーが出る場合は, このセルをもう一度実行してください
E = Encoder(**E_args).to(device)
D = Decoder(**D_args).to(device)

optimizer_E = optim.Adam(E.parameters())
optimizer_D = optim.Adam(D.parameters())

#### 2.3.3. その他の関数

In [ ]:
def sample_z_prior(batch_size):
    """事前分布p(z)からzをサンプリングする
    :param batch_size: int, バッチサイズ
    :return z: tensor, サンプリングされたz, size=(バッチサイズ, latent_size)
    """
    z = torch.randn(batch_size, latent_size).to(device)
    return z

### 3. 学習

#### 3.1. 言語モデルの評価

負の対数尤度及びPerplexityにより行います.
ともに値が低いほど良い言語モデルと言えます.

##### 負の対数尤度
$$
\begin{align}
    \mathrm{NLL}(x) &= -\log \prod^N_{n=1}p_{\theta} (x_n|x_{<n}, z)
\end{align}
$$

##### Perplexity

$$
\begin{align}
    \mathrm{PPL}(x) &= \left(\prod^N_{n=1} p_{\theta}(x|z)\right)^{-\frac{1}{N}} \\
    &= \left(e^{-\mathrm{NLL}(x)}\right)^{-\frac{1}{N}} \\
    &= e^{\frac{\mathrm{NLL}(x)}{N}}
\end{align}
$$

In [ ]:
def compute_loss_vae(x, x_lens, lmd, is_train=False):
    """VAEの損失関数 (負の対数尤度 + lmd * KLダイバージェンス) を計算する
    :param x: tensor, 入力単語id列のバッチ, size=(バッチサイズ, 系列長)
    :param x_len: tensor, 入力単語の系列長のバッチ, size=(バッチサイズ, 系列長)
    :param lmd: int, KLダイバージェンスの大きさをコントロールする係数
    :param is_train: bool, モデル (EとD) のパラメータを更新するか否か
    :return loss_vae: tensor, VAEの損失, size=()
    :return nll: tensor, 負の対数尤度, size=()
    :return loss_kl: tensor, KLダイバージェンス, size=()
    :return ppl: tensor, パープレキシティ, size=()
    """
    
    # Encoder
    input_encoder = x[:, :-1] # [<S>, x_1, ..., x_T]
    input_encoder_lens = x_lens - 1
    z, mean, lvar, loss_kl = E.forward(input_encoder, input_encoder_lens)
    
    # Decoder
    input_decoder = x[:, :-1] # [<S>, x_1, ..., x_T]
    target_decoder = x[:, 1:].contiguous() # [x_1, x_2, ..., </S>]
    output_decoder = D.forward(input_decoder, z)
    
    # 損失
    nll_all = F.cross_entropy(output_decoder.view(-1, vocab_size), target_decoder.view(-1), ignore_index=PAD, size_average=False, reduce=False)
    nll_mb = torch.sum(nll_all.view(output_decoder.size(0), output_decoder.size(1)), dim=1)
    nll = nll_mb.mean()
    ppl_mb = torch.exp(nll_mb / input_encoder_lens.type(torch.float))
    ppl = ppl_mb.mean()
    
    loss_vae = nll + lmd * loss_kl
    
    if is_train:
        E.zero_grad()
        D.zero_grad()
        loss_vae.backward()
        optimizer_E.step()
        optimizer_D.step()
    
    return loss_vae, nll, loss_kl, ppl

#### 3.2. 学習

In [ ]:
step = 0
start_time = time.time()
for epoch in range(num_epochs):
    # Train
    E.train()
    D.train()
    nll_train = []
    kl_train = []
    ppl_train = []
    for batch_x, batch_x_lens in dataloader_train:
        lmd = get_kl_weight(step)
        
        # 損失の計算 & パラメータの更新
        loss, nll, loss_kl, ppl = compute_loss_vae(batch_x, batch_x_lens, lmd, is_train=True)
        nll_train.append(nll.item())
        kl_train.append(loss_kl.item())
        ppl_train.append(ppl.item())
        
        step += 1
    
    # Valid
    E.eval()
    D.eval()
    nll_valid = []
    kl_valid = []
    ppl_valid = []
    for batch_x, batch_x_lens in dataloader_valid:
        # 損失の計算
        loss, nll, loss_kl, ppl = compute_loss_vae(batch_x, batch_x_lens, lmd, is_train=False)
        nll_valid.append(nll.item())
        kl_valid.append(loss_kl.item())
        ppl_valid.append(ppl.item())

    print('EPOCH: {}, LMD: {:.2f}, Train [NLL: {:.3f}, KL: {:.3f}, PPL: {:.3f}], Valid [NLL: {:.3f}, KL: {:.3f}, PPL: {:.3f}], Elapsed Time: {:.2f}[s]'.format(
        epoch + 1,
        lmd,
        np.mean(nll_train),
        np.mean(kl_train),
        np.mean(ppl_train),
        np.mean(nll_valid),
        np.mean(kl_valid),
        np.mean(ppl_valid),
        time.time() - start_time
    ))

### 4. 生成

#### 4.1. ランダム

事前分布$p(z)$からサンプリングした$z$により文を生成してみましょう.

In [ ]:
def sample(batch_size, max_length):
    """Decoderから文を生成する
    :param batch_size: int, バッチサイズ
    :param max_length: int, 生成文のmax長
    :return x_hat: tensor, 生成文, size=(バッチサイズ, 系列長)
    """
    z = sample_z_prior(batch_size)
    x_hat = D.sample(z, max_length)
    
    return x_hat

In [ ]:
n_samples = 30
max_length = 20

x = sample(n_samples, max_length)
x = x.cpu().numpy()

for i, x_i in enumerate(x):
    x_i = ' '.join([vocab.id2word[i] for i in x_i])
    x_i = re.sub(r' {}.*'.format(EOS_TOKEN), '', x_i) # </S>以降は除去
    print('{}. {}'.format(i, x_i))

#### 4.2. Interporation

事前分布$p(z)$からサンプリングしたデータ間を遷移したときの生成文の変化を確認してみましょう.

In [ ]:
n = 10

z1 = sample_z_prior(1).squeeze()
z2 = sample_z_prior(1).squeeze()

interpolation = torch.zeros((latent_size, n)).to(device)

for i, (z1_i, z2_i) in enumerate(zip(z1, z2)):
    interpolation[i] = torch.linspace(z1_i, z2_i, n).to(device)

interpolation.t_()

x_hat = D.sample(interpolation, max_length).cpu().numpy()

for i, x_i in enumerate(x_hat):
    x_i = ' '.join([vocab.id2word[i] for i in x_i])
    x_i = re.sub(r' {}.*'.format(EOS_TOKEN), '', x_i) # </S>以降は除去
    print('{}. {}'.format(i, x_i))

参考資料

- 原論文: http://www.aclweb.org/anthology/K16-1002
- Neural Networks for NLP - Latent Random Variable - (CMU): http://www.phontron.com/class/nn4nlp2018/schedule/latent-random-variables.html
- PyTorch実装 (非公式) : https://github.com/timbmg/Sentence-VAE
- TensorFlow実装 (非公式) : https://github.com/ryokamoi/original_textvae